## Описание проекта
Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
# 
import pandas as pd
import numpy as np
#
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import GaussianNB  
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix,r2_score,roc_curve,f1_score,\
mean_squared_error,roc_auc_score,recall_score,precision_score,make_scorer,precision_recall_curve,mean_absolute_error
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold,RandomizedSearchCV,cross_validate,\
cross_val_score,cross_val_predict
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,LinearRegression,LassoCV,RidgeCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder,OrdinalEncoder
from sklearn import model_selection, svm
from sklearn.utils import resample

from IPython.display import display
from joblib import dump,load
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats as st
import random
from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

import random

import warnings
warnings.filterwarnings("ignore")

import time
import calendar

In [2]:
# 2 варианта чтобы открыть файл:
# сразу будем создавать time index,так удобнее сортировать данные в дальнейшем
try:
    df = pd.read_csv('/datasets/insurance.csv')
except FileNotFoundError:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')
    #print('FileNotFoundError')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
#функция дает первичное представление об исходных данных: типы данных,количество пропусков,кол-во дубликатов,
#хар-ки числовых столбцов 
def df_info(df):
    print(80*"+")
    #print(f'Название таблицы : {df.name}')
    #print(80*"+")
    df.info()
    print(80*"+")
    print('Описание количественных переменных:')
    print(df.describe(percentiles=[.25,.5]))
    print(80*"+")
    print(f'количество дубликатов в таблице = {df.duplicated().sum()}')

In [4]:
df_info(df)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Описание количественных переменных:
               Пол      Возраст      Зарплата  Члены семьи  Страховые выплаты
count  5000.000000  5000.000000   5000.000000  5000.000000        5000.000000
mean      0.499000    30.952800  39916.360000     1.194200           0.148000
std       0.500049     8.440807   9900.083569     1.091387           0.463183
min       0.000000    18.000000   5300.000000     0.000000           0.000000
25%       0.000000    24.000000  33300.000000

In [5]:
# посмотрим примеры строк с полным повторениями,т.е. дублями
# отсортируем по Зарплате, так легче увидеть повторы

df.loc[df.duplicated(keep=False)].sort_values(by='Зарплата').head(10)

#pd.concat(i for _,i in df.loc[df.duplicated(keep=False)].groupby("Зарплата") if len(i) > 1)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2955,1,32.0,21600.0,0,0
2988,1,32.0,21600.0,0,0
361,0,50.0,24700.0,1,2
2869,0,50.0,24700.0,1,2
333,0,32.0,25600.0,1,0
4230,0,32.0,25600.0,1,0
1378,0,36.0,26400.0,0,0
2723,0,36.0,26400.0,0,0
1002,1,34.0,26900.0,0,0
1140,1,34.0,26900.0,0,0


In [6]:
# удалил дубликаты!
df = df.drop_duplicates()
df_info(df)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  4847 non-null int64
Возраст              4847 non-null float64
Зарплата             4847 non-null float64
Члены семьи          4847 non-null int64
Страховые выплаты    4847 non-null int64
dtypes: float64(2), int64(3)
memory usage: 227.2 KB
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Описание количественных переменных:
               Пол      Возраст      Зарплата  Члены семьи  Страховые выплаты
count  4847.000000  4847.000000   4847.000000  4847.000000        4847.000000
mean      0.498453    31.023932  39895.811842     1.203425           0.152259
std       0.500049     8.487995   9972.953985     1.098664           0.468934
min       0.000000    18.000000   5300.000000     0.000000           0.000000
25%       0.000000    24.000000  33200.000000

### Подитог:
- загрузил данные;
- удалил 153 строки дубликатов, точно не пригодятся эти данные;
- т.к. в данном проекте не требутеся аналитика/предобработка данных, но судя по describe данные хорошие, пропусков нет, выбросов тоже.

## Умножение матриц

### Ответьте на вопрос и обоснуйте решение.
Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
- Изменится. Приведите примеры матриц.
- Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

In [7]:
# разделим данные на признаки и таргет
X = df.drop(labels = "Страховые выплаты",axis = 1)
y = df["Страховые выплаты"]

In [8]:
# проверим полученные размерности
print(X.shape,y.shape)

(4847, 4) (4847,)


In [9]:
# добавим 1 вектор, что соответствует w0
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
X.shape

(4847, 5)

<font color='green'>  
**UPD**:  

Создададим различные варианты матриц для дальнейшего домножения на X.  
Также посмотрим детерминант матриц , что будет подтверждать обратима или необратима матрица

</font>

Обратимые матрицы:  

$X$ - исходная матрица;  
$E$ - единичная матрица;  
$R$ - рандомная(обратимая) матрица;  
$Q$ - матрица полученная $X.T @ X$  

Необратимая матрица:  
$M$ - матрица со всеми 0.


все матрицы имеют размерность равную n x n.  
где n = длина вектора признаков, т.е. 5

В задании простя домножить Признаки на обратимую матрицу, т.е. домножить справа!

In [10]:
# функция проверяет матрицу на обратимость, высчитывает определитель 
def error_inv(X):
    try: 
        np.linalg.inv(X)
        print("Матрица обратима, детерминант = ",np.linalg.det(X))
    except np.linalg.LinAlgError:
        try:
            print("Матрица необратима, детерминант = ",np.linalg.det(X))
        except:
            print("Матрицы не удовлетваоряет требования к обратимой матрице, возможно неквадратная!")

In [11]:
error_inv(X)

Матрицы не удовлетваоряет требования к обратимой матрице, возможно неквадратная!


In [12]:
E = np.eye(5)
E

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [13]:
error_inv(E)

Матрица обратима, детерминант =  1.0


In [14]:
X_E = X.dot(E)
X_E.shape

(4847, 5)

In [15]:
np.random.seed(0)
R = []
for i in range(5):
    vector = np.random.rand(5)
    R.append(vector)
    
R = np.array(R)
R

array([[0.5488135 , 0.71518937, 0.60276338, 0.54488318, 0.4236548 ],
       [0.64589411, 0.43758721, 0.891773  , 0.96366276, 0.38344152],
       [0.79172504, 0.52889492, 0.56804456, 0.92559664, 0.07103606],
       [0.0871293 , 0.0202184 , 0.83261985, 0.77815675, 0.87001215],
       [0.97861834, 0.79915856, 0.46147936, 0.78052918, 0.11827443]])

In [16]:
error_inv(R)

Матрица обратима, детерминант =  0.009658225505885123


In [17]:
X_R = X.dot(R)
X_R.shape

(4847, 5)

In [18]:
Q = X.T @ X
Q 

array([[4.84700000e+03, 2.41600000e+03, 1.50373000e+05, 1.93375000e+08,
        5.83300000e+03],
       [2.41600000e+03, 2.41600000e+03, 7.49940000e+04, 9.67618000e+07,
        2.88800000e+03],
       [1.50373000e+05, 7.49940000e+04, 5.01429700e+06, 5.99212070e+09,
        1.80553000e+05],
       [1.93375000e+08, 9.67618000e+07, 5.99212070e+09, 8.19683486e+12,
        2.31029800e+08],
       [5.83300000e+03, 2.88800000e+03, 1.80553000e+05, 2.31029800e+08,
        1.28690000e+04]])

In [19]:
error_inv(Q)

Матрица обратима, детерминант =  5.771461849364651e+27


In [20]:
X_Q = X.dot(Q)
X_Q.shape

(4847, 5)

In [21]:
M = np.copy(R)
M[:,:] = 0
M

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [22]:
error_inv(M)


Матрица необратима, детерминант =  0.0


In [23]:
X_M = X.dot(M)
X_M.shape

(4847, 5)

Свойства обратной матрицы:  
2. Матрица $А$ называется обратной, если существует матрица $В$, удовлетворяющая выражению:
$ AB = BA = E $,
где $E$ - единичная матрица.
$А$ и $В$ называются взаимно обратными
3. Если матрица $А$ обратима, то для нее существует только **одна** матрица обратная к $А$.Обратная матрица обозначается как $A^-1$  
4. Если матрица $А$ обратима , то матрица $A^-1$ также является обратимой  
5. Произведение любого числа обратимых матриц дает обратимую матрицы  
6. Определить необратимой матрицы $= 0$, определитель обратимой матрицы $\not= 0$  
7. $(A.T)^-1$ = $(A^-1).T$

Доказательство п.3:  
Предположим матрица $A$ имеет 2 обратных матрицы $B$ и $C$, тогда:  
    
$ AC = E = BA \Rightarrow $   
$ B = BE = B(AC)= (BA)C = EC = C \Rightarrow$     
$ B = C $   
</font>


**Обозначения:**

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $w*$ - вектор весов линейной регрессии матрицы полученной домножение исходной на обратимую матрицу(нулевой элемент равен сдвигу)

- $\hat{y}$ - предсказания по исходной матрице признаков

- $\hat{y}*$ - предсказания по матрице полученной домноженной на обратимую матрицу

**Основные свойства обратной матрицы:**

- $$det(A) = \frac{1}  {det(A)} $$
- $$(AB)^{-1} = B^{-1}A^{-1}$$ для двух квадратных обратимых матриц $A$ и $B$
- $$(A^T)^{-1} = ((A)^{-1})^T$$




**Основные свойства единичной матрицы:**
- $$AE = EA = A$$
- $$AA^{-1}=E$$


**Сочетательное свойство матриц, ассоциативность:**
- $$A(BC) = (AB)C$$



<h4><center>Исходная матрица признаков $X$</center></h4>  
  
Предсказания:


$$
\hat{y} = Xw
$$


Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Распишем формулу предсказания:
$$
\hat{y} = X (X^T X)^{-1} X^T y
$$

- раскрываем скобки:
$$
\hat{y} = X (X)^{-1} (X^T)^{-1} X^T y
$$

- по свойству единичной матрицы получаем:
$$
\hat{y} = E E y
$$

- или 
$$
\hat{y} = y
$$

Доказали формулу для вычисления предсказаний.

<h4><center>Новая матрица признаков $X.dot(P)$</center></h4>  


Предсказания:

$$
\hat{y}* = XPw*
$$

Формула обучения:

$$
w* = ((XP)^T XP)^{-1} (XP)^T y
$$


Раскроем транспонирование матриц:
$$
w* = (P^T X^T XP)^{-1} P^TX^T y
$$

Раскроем скобки для произведения обратных матриц:
$$
w* = (XP)^{-1}(P^T X^T)^{-1} P^TX^T y
$$

Еще раз раскроем скобки для произведения обратных матриц:
$$
w* = P^{-1}X^{-1}(X^T)^{-1}(P^T)^{-1}P^TX^T y
$$

Сокращаем произведение матриц $(P^T)^{-1}P^T$ по свойству единичной матрицы. Получаем $E$ или $1$, т.е. можно опустить ;
$$
w* = P^{-1}X^{-1}(X^T)^{-1}X^T y
$$
или
$$
w* = P^{-1}(X^TX)^{-1}X^T y
$$

Распишем предсказания по новым признакам с учетом полученной формулы для $w*$:


$$
\hat{y}* = XPP^{-1}(X^TX)^{-1}X^T y
$$


Сокращаем произведение матриц $P(P)^{-1}$ по свойству единичной матрицы. Получаем $E$ или $1$, т.е. можно опустить ;
$$
\hat{y}* = X(X^TX)^{-1}X^T y
$$

**Получили формулу для $\hat{y}*$ которая полностью совпадает с формулой для $\hat{y}$**

Из этого можно сделать вывод, что независимо от того, какую метрику будем использовать MSE,MAE или R2 **значения метрик не изменятся!**

Также из формул выше видно, что коэф.исходной и домноженной матриц связаны как: $w* = P^{-1}w$


Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Формула MSE согласно sklearn:
$${MSE}(y, \hat{y}) = \frac{1}{n_\text{samples}} \sum_{i=0}^{n_\text{samples} - 1} (y_i - \hat{y}_i)^2.$$

Формула MAE согласно sklearn:  
$$ {MAE}(y, \hat{y}) = \frac{1}{n_{\text{samples}}} \sum_{i=0}^{n_{\text{samples}}-1} \left| y_i - \hat{y}_i \right|.$$

**Примеры полученных метрик при перемножении исходной матрицы на обратимые и необратимую матрицы:**

In [24]:
def metrics(X,y,name):
    model = LinearRegression()
    model.fit(X,y)
    predict = model.predict(X)
    return print("Метрика MSE для матрицы",name,"равна",mean_squared_error(y,predict))

In [25]:
metrics(X,y,name = 'X')

Метрика MSE для матрицы X равна 0.125272638266816


In [26]:
metrics(X_E,y,name = 'X_E')

Метрика MSE для матрицы X_E равна 0.125272638266816


In [27]:
metrics(X_R,y,name = 'X_R')

Метрика MSE для матрицы X_R равна 0.1252724079401175


In [28]:
metrics(X_Q,y,name = 'X_Q')

Метрика MSE для матрицы X_Q равна 0.12527264044835446


In [29]:
metrics(X_M,y,name = 'X_M')

Метрика MSE для матрицы X_M равна 0.2198540875868632


По результатам видно, что метрика нулевой (необратимой матрицы) значительно хуже метрик для обратимых матриц и исходной матрицы.  
Метрики обратимых матриц и исходной сопадают, не изменились 
**UPD (в пределах погрешности для типа данных float)**


In [30]:
import sys
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)


**Ответ:** качество линейной регрессии **не изменится**

**Обоснование:**   
- выше приведено доказательство того, что предсказания исходной матрицы = предсказаниям исходной матрицы домноженной на обратимую матрицу

## Алгоритм преобразования

### Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

Основная задача проекта защитить данные клиентов страховой компании:
- разработать алгоритм преобразования данных, чтобы по ним было сложно восстановить персональную информаци;
- обосновать корректность его работы.

**Алгоритм и обоснование алгоритма**:

### Вариант решения №1:
- необходимо домножить каждую строку (вектор) признаков на обратимую матрицу. Как было доказано выше метрика MSE как и R2_score не изменится,т.к. результы предсказаний до умножения и после идентичны.  
**UPD** Такая матриц уже была сформирована ранее ($X\_R$ = $X@R$ , где $R$ - обратимая матрица и получена рандомно).

Т.к. матрица $X$_$R$ уже существует, то домножать повторно не имеет смысла, просто выведем значение матрицы $X$_$R$ ниже:

In [31]:
X_R

array([[ 4356.24731772,  1026.46913989, 41323.19018192, 38636.8133844 ,
        43156.44040217],
       [ 3348.86017226,   794.14261697, 31666.74842337, 29613.85939382,
        33064.27122128],
       [ 1853.22413334,   440.63948829, 17502.09281215, 16368.67895564,
        18272.73881367],
       ...,
       [ 2972.02381083,   698.29507812, 28238.6993774 , 26400.1317315 ,
        29495.49275038],
       [ 2870.67661372,   676.3275368 , 27242.04490423, 25469.93901557,
        28452.12196055],
       [ 3561.79119491,   837.62792897, 33822.2269927 , 31621.36986956,
        35325.40759919]])

In [32]:
error_inv(R)

Матрица обратима, детерминант =  0.009658225505885123


In [33]:
error_inv(X_R)

Матрицы не удовлетваоряет требования к обратимой матрице, возможно неквадратная!


### Вариант решения №2:
- необходимо применить метод StandardScaler.
Суть данного метода описывается формулой
$$z = \frac{(x - u)} {s}$$

$x$ - признак, который необходимо изменить  
$u$ - среднее значение столбца данного признака  
$s$ - стандратное отклонение(СКО) признака

Данный метод приведет все признаки в диапазон 0 $+-$ одно стандартное отклонение

In [34]:
# функция стандартизирует данные аналогично метода StandardScaler
# т.к. в некоторых ячейках numpy определяет значения как Nan, применил метод np.nan_to_num.(меняет на 0)

def my_ss(X):
    s = np.std(X,axis = 0)
    u = np.mean(X,axis = 0)
    X_new = []
    for i in range(X.shape[0]):
        vector = []
        for j in range(X.shape[1]):
            ss = (X[i,j] - u[j])/s[j]
            ss = np.nan_to_num(ss)
            vector.append(ss)
        X_new.append(vector)
    return X_new    

In [35]:
X_my_ss = np.array(my_ss(X))
X_my_ss

array([[ 0.        ,  1.0030995 ,  1.1754362 ,  0.97315092, -0.18517565],
       [ 0.        , -0.99691008,  1.76456423, -0.19011493, -0.18517565],
       [ 0.        , -0.99691008, -0.23847105, -1.89490109, -1.09546611],
       ...,
       [ 0.        , -0.99691008, -1.29890149, -0.60126924,  0.7251148 ],
       [ 0.        ,  1.0030995 , -1.06325028, -0.72160708,  1.63540526],
       [ 0.        ,  1.0030995 , -0.35629665,  0.07061707, -0.18517565]])

In [36]:
ss = StandardScaler()
X_ss = ss.fit_transform(X)

X_ss

array([[ 0.        ,  1.0030995 ,  1.1754362 ,  0.97315092, -0.18517565],
       [ 0.        , -0.99691008,  1.76456423, -0.19011493, -0.18517565],
       [ 0.        , -0.99691008, -0.23847105, -1.89490109, -1.09546611],
       ...,
       [ 0.        , -0.99691008, -1.29890149, -0.60126924,  0.7251148 ],
       [ 0.        ,  1.0030995 , -1.06325028, -0.72160708,  1.63540526],
       [ 0.        ,  1.0030995 , -0.35629665,  0.07061707, -0.18517565]])

In [37]:
# кажется что функция работает аналогично StandardScaler, но значительно медленнее))
X_ss == X_my_ss

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       ...,
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

## Проверка алгоритма

In [38]:
# попробовал создать алгоритм для линейной регрессии
X.dot(np.linalg.inv(X.T.dot(X))).dot(X.T).dot(y)
# результаты совпадают с результатами из Sklearn


array([ 0.51932881,  0.69230097,  0.09412742, ..., -0.26164511,
       -0.19394186,  0.05044763])

In [39]:
model = LinearRegression()
model.fit(X,y)
predict = model.predict(X)
predict

array([ 0.51932881,  0.69230097,  0.09412742, ..., -0.26164511,
       -0.19394186,  0.05044763])

In [40]:
model = LinearRegression()
model.fit(X,y)
predict = model.predict(X)

print("R2_score для исходной матрицы = ",r2_score(y,predict))

R2_score для исходной матрицы =  0.4302010044852068


In [41]:
model = LinearRegression()
model.fit(X_R,y)
predict = model.predict(X_R)

print("R2_score для исходной матрицы домноженной на обратимую матрицу = ",r2_score(y,predict))

R2_score для исходной матрицы домноженной на обратимую матрицу =  0.43020205211957674


In [42]:
model = LinearRegression()
model.fit(X_my_ss,y)
predict = model.predict(X_my_ss)

print("R2_score для матрицы после применения  функции my_ss (StandardScaler) = ",r2_score(y,predict))

R2_score для матрицы после применения  функции my_ss (StandardScaler) =  0.4302010044852068


## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования